# KAUST bioinformatics workshop
### By Chris Hempel (christopher.hempel@kaust.edu.sa)

Follow along by copying the code into your IBEX terminal.

## 1. Setup

### 1.1 Working directory

Make sure you're in your home directory:

`cd ~`

Then download the GitHub directory for this workshop to set up a working directory. All code will be run and programs will be installed in this directory:

`git clone https://github.com/hempelc/kaust_bioinfo_workshop.git`

### 1.2 Program installations

Generate and navigate to a directory in the working directory called "programs":

`mkdir ~/kaust_bioinfo_workshop/programs`

`cd ~/kaust_bioinfo_workshop/programs`

#### 1.2.1 Installing mamba

Mamba allows you to generate environments in which you can install packages (=programs). That way, you don't mix up the requirements for different programs you want to run. We install mamba following the instructions [here](https://mamba.readthedocs.io/en/latest/installation/mamba-installation.html):

`wget "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh"`

`bash Miniforge3-$(uname)-$(uname -m).sh`

`rm Miniforge3-$(uname)-$(uname -m).sh`

#### 1.2.2 Installing FastQC and MultiQC using mamba

You can find instructions on how to install certain packages using the [anaconda search engine](https://anaconda.org/).
For example, multiqc can be installed as shown [here](https://anaconda.org/bioconda/multiqc).

We create a new mamba environment using `mamba create` and give it a name with the `-n` parameter. Then we state what packages we want to install. In this case, we want to install fastqc and multiqc (to determine the quality of our sequences, more on that later), which are both found on the package installation channel bioconda:

`mamba create -n multiqc bioconda::fastqc bioconda::multiqc`

Alternatively, we can use the parameter `-c` to state that mamba should check the channel bioconda for the installation:

`mamba create -n multiqc -c bioconda fastqc multiqc`

#### 1.2.3 Installing Apscale using a combination of mamba and GitHub

[Apscale](https://github.com/DominikBuchner/apscale) is a very fast and easy-to-use metabarcoding processing pipeline. We will later use it to process some sequencing data. We will install a modified version of Apscale that I generated, as well as a wrapper for Apscale to easily run it in the terminal.

First, we will set up a mamba environment in which we will install all required packages/programs for Apscale. We could type all programs into one command to install them in the environment, but we need to install a lot of them with specific versions, which is a lot to type. Therefore, we will use a .yml file for the installation instead. The .yml file contains all information on packages to be installed, including versions, and can be read by mamba. This ensures that all of us install exactly the same version of the Apscale mamba environment.

Clone the GitHub directory containing my modified version of Apscale to retrieve the required .yml file:

`git clone https://github.com/hempelc/apscale.git`

Create the mamba env directly from the .yml file:

`mamba env create -f apscale/conda-env_apscale_IBEX.yml`

Now, all required packages for Apscale are installed, just Apscale itself is missing. To install Apscale, we activate the environment and install the modified version of Apscale with the setup.py file located in the cloned GitHub directory:

`mamba activate apscale`

`cd apscale`

`python setup.py install`

`cd ~/kaust_bioinfo_workshop/programs`

Deactivate the environment again. I'd recommend activating mamba environments only before you need them, just to make sure you don't accidentally install something into your environment and mess it up:

`mamba deactivate`

Lastly, we need to install the Apscale wrapper. Therefore, clone another of my GitHub directories:

`git clone https://github.com/hempelc/apscale_wrapper.git`

And make the wrapper scripts available in your PATH (otherwise they won't work):

`echo -e '\n#Manually exported paths\nexport PATH=$PATH:/home/'"$(whoami)"'/programs/apscale_wrapper' >> ~/.bash_profile && source ~/.bash_profile` 

#### 1.2.4 Installing Krona

[Krona](https://github.com/marbl/Krona/wiki) allows you to generate great interactive visualizations for metabarcoding data. The visualizations can be used for publications, but their main use it to interactively explore the data.

Create a new mamba environment for krona:

`mamba create -n krona bioconda::krona`

Activate the environment and run a script that came with krona to install a krona taxonomy database:

`mamba activate krona && ktUpdateTaxonomy.sh && mamba deactivate krona`

#### 1.2.5 Installing Cyberduck

On your local computer (not on IBEX!): download and install [Cyberduck](https://cyberduck.io/download/). Cyberduck allows you to connect to IBEX and view and download your files with a graphic user interface, just like you're used to!

## 2. Download sequencing data (proudly sponsored by Ken) and the MIDORI2 COI BLAST database

Move back the working directory:

`cd ~/kaust_bioinfo_workshop`

All data we need is accessible on my Google Drive. You can download it using the following command:

`wget --header="Host: drive.usercontent.google.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7" --header="Accept-Language: en-US,en;q=0.9" --header="Cookie: CONSENT=PENDING+238; SOCS=CAESHAgCEhJnd3NfMjAyNDAxMDItMF9SQzQaAmVuIAEaBgiAmfKsBg; SID=fAgtNWQlH3PTQE33XKx_z5pNwzPj-pde5PGFKH1_FIlUiK_-uzzZnxsd_rE5mCEp83FqVg.; __Secure-1PSID=fAgtNWQlH3PTQE33XKx_z5pNwzPj-pde5PGFKH1_FIlUiK_-l8agA_b1J4uQawJ8DG1P3w.; __Secure-3PSID=fAgtNWQlH3PTQE33XKx_z5pNwzPj-pde5PGFKH1_FIlUiK_--iPUnxAdhQEYBdcMVOfVWg.; HSID=AY895b6yQh3ERBnKf; SSID=AYo1w_ypkxdWNJZp6; APISID=zW20S9QhUHd5O6BQ/A3ahD13pc-c7fiu5h; SAPISID=vInNTptJkQe-o4ZR/A-gXcuimBrHpwkjBL; __Secure-1PAPISID=vInNTptJkQe-o4ZR/A-gXcuimBrHpwkjBL; __Secure-3PAPISID=vInNTptJkQe-o4ZR/A-gXcuimBrHpwkjBL; AEC=Ae3NU9Pcnwo7fX1Y44YCSH1DD97iIkOTQ1hF5mcn6RcSB4GoNQsYkezF1d8; NID=511=N5eBkHK_g_3jKNGBOSAbaWKR8uU3gfv7-Yj6zFcKYxwq8yifhRWkGillvUx76Ym9qlaoosfzj0M8vDDE9chSGcMpFkyLSJ0kTeskiihIjjbxXpR1XEs1DNuHHb4EngByMLdOH3T2_CbOJwjc4UkDNx5j8ezCJbJ-Xo4Vj80bXegGkJeW-mMeh0RPHJej_iaDllBGfgBZmK65rytVId-6ZmzZw3fjtUDX9w6ROJet4udX7Y_v8cfMzpr8_VGtWMNEwF8UAEENQKLEIRsSFO6qsCMbPkwkGN3ozjwAjhxDFtfOP60hQlBDB07o44gwlZt88w_jr1YS0eZrytbaLhyRaSMiv3JtB7o4Tfy_iD3cMWhUX9Tah6CTIZnKLDIbZQwM4ofosoAho-WsQ8JFOvDqSkTayID5MyXJsNax7PmsJiHzIkM3F7vun-B1Bxv_EW5Hfqg; 1P_JAR=2024-01-19-23; __Secure-ENID=17.SE=phTR0YJslGNoHRdC7V5qpmCYazlGyDzZNTty0q6xooH5qAdxe8wNArCqPTs8xnX_kEjamFr5q6rKakaFNPzLN8PYUyaCT6T4-1QFMJ0RHM0b-0dAmCgxC9oiFzW7nnOrxVQvyNU7mamZKxiimTraF1Wri6U9beW0J8uTsnVlOVUv0q2KGgi67dYqjD7hh9AoLt0aaZDuyr9aV2tYQqq8Fj6kOV3Aej-Tfj5gImFIwhnrihjTWLE5dmMqHxIapt0BlKG4wOtHh4Sb; __Secure-1PSIDTS=sidts-CjIBPVxjSvc6nIvT0_2MEjKb0q6VKdKIxwX8n9OtTaQXMSj3VwmSHOUaxVsPUf-Wnwho2RAA; __Secure-3PSIDTS=sidts-CjIBPVxjSvc6nIvT0_2MEjKb0q6VKdKIxwX8n9OtTaQXMSj3VwmSHOUaxVsPUf-Wnwho2RAA; SIDCC=ABTWhQGtYTjREbjYswNUUslB4UhhMB1mprMRB1EqqPb_3Fm0OnRQQZPVpuBLRuoSmOutXtP0yIiX; __Secure-1PSIDCC=ABTWhQHTTNCTiznY3ld2rbu1LtLi6rpBLOZNbmrlLbydOfZnE0QGG0Hv_i1f-1VTiBTKa9jq4zk; __Secure-3PSIDCC=ABTWhQGUR5aPEShdmYCFnHgbBIbhNEAY_ks4vJmpFMri5hkoQMxvfZXog_6ZfhH1X-RCKzQA5ls" --header="Connection: keep-alive" "https://drive.usercontent.google.com/download?id=1134kQbJC76VyXe8Kdy-MfE_XSG1KIA7f&export=download&authuser=0&confirm=t&uuid=73de8872-8f7e-4950-9e4f-d97222fc0be4&at=APZUnTWBelFkm3Ab6e2cHF3AflDn:1705708397053" -c -O 'kaust_bioinfo_workshop_files.tar.gz'`

Lastly, uncompress it so that we can access the files:

`tar -xzvf kaust_bioinfo_workshop_files.tar.gz`


`rm kaust_bioinfo_workshop_files.tar.gz`

## 3. Analyze the quality of the sequencing data using MultiQC

Let's have a look at the quality of Ken's sequencing data (I just included a few samples). We will use the programs FastQC and MultiQC to get an overview of the quality. We installed both programs in a mamba environment, so we just need to activate the environment to be able to use the programs:

`mamba activate multiqc`

First, generate a directory for the output of FastQC. Then, run FastQC on the samples and save the output in the generated output directory. Note: we don't run this command with SBATCH because it's a very small, simple job:

`mkdir fastqc_output`

`fastqc kaust_bioinfo_workshop_files/example_sequences/* -o fastqc_output`

Then run MultiQC on the FastQC output:

`multiqc --interactive -o multiqc_output fastqc_output`

The quality summary is saved under `multiqc_output/multiqc_report.html`. It's an HTML file so we will have to download it to our local computer and open it in a web browser. Open Cyberduck, download the MultiQC HTML file to your local computer, and let's check out the file together. But before we do that, don't forget to deactivate your mamba environment:

`mamba deactivate`

## 4. Process the sequencing data with Apscale 

When we ran FastQC and MultiQC above, we just ran them in the login terminal of IBEX because they're small jobs. However, running Apscale is not a small job, as it requires GBs of RAM and multiple cores (the more cores, the faster it runs). Therefore, to run Apscale, we make use of IBEX' computational power by submitting a script to the job scheduler (SLURM). In the script, we list our commands to run Apscale. Then, SLURM will send our code to one of the many powerful computers within the IBEX computer cluster and run the code there. The results of the run are sent back to our user account, specifically to the location where we started the run.

To submit a job via SLURM, we need to configure our script in a specific format. You will find a pre-formatted script to run Apscale in the working directory. Open it with the following command and let's have a look at it together:

`cat kaust_bioinfo_workshop_apscale_sbatch.sh`

You will notice a bunch of parameters for our Apscale run. To find more information on these parameters, you can open the help page of the Apscale wrapper with the following command (note: we activate the mamba Apscale environment to run the help command and deactivate it after):

`mamba activate apscale && apscale_wrapper.py --help && mamba deactivate`

Now, let's submit the SLURM script to the job scheduler using the command sbatch:

`sbatch kaust_bioinfo_workshop_apscale_sbatch.sh`

You can find additional information on your run via the sacct command, which displays all your submitted runs as well as their used RAM, CPU time, runtime, and more (note: I modified the command to separate runs by lines):

`term_width=$(tput cols) && line="" && for ((i=0; i<term_width; i++)); do line+="-"; done && sacct --units G --format="JobID,JobName%30,start%10,ncpus%5,MaxRSS%7,time,Elapsed,mincpu,state" --starttime 2023-05-01 | sed "s/extern.*$/&\n${line}/"`

You can also check what your run is currently doing by viewing the log file that is generated for the run. The log file contains all text output that is generated by your script:

`cat kaust_bioinformatics_workshop_apscale.*.log`

Once the run is done, let's investigate the files generated by Apscale by downloading them to our local computer using Cyberduck. Simultaneously, download [this](https://github.com/hempelc/kaust_bioinfo_workshop/blob/main/metabarcoding_overview.pptx) PowerPoint presentation to learn about the different metabarcoding processing steps while we're investigating Apscale's output.

## 5. EXTRA: Setting up a BLAST database from scratch

We used a curated BLAST database to annotate our OTU sequences, specifically [MIDORI2](https://www.reference-midori.info/), which curates sequences from NCBI for many metabarcoding target genes. Generally, when processing metabarcoding data, such curated databases are the way to go, as they are more reliable and targeted than assembling a database from scratch.

However, there are cases where it is beneficial or necessary to generate a BLAST database yourself. Let's go through the required steps.

First, you need a FASTA file containing the sequences you want to add to your database. A good starting point is going to [NCBI GenBank](https://www.ncbi.nlm.nih.gov/genbank/) and searching for sequences for your target organism(s) and gene(s). 

In Ken's data, most of the sequences are assigned to an OTU belonging to the family "Myctophidae". Since his data is a gut-content sample, this OTU should stem from his fish host. Let's see how good of a match we get when BLASTing this OTU against all Phosichthyidae COI sequences on NCBI GenBank.

In the search field of [NCBI GenBank](https://www.ncbi.nlm.nih.gov/genbank/), search for <u>**"Myctophidae" AND "COI"**</u>. At the point of writing this tutorial, there are 265 hits=sequences. Let's download all of them in one FASTA file and upload them to IBEX with Cyberduck. I already did this for you, so you will find the FASTA file located in the workshop files, but if you wanted to do this yourself, you would click on "Send to", "Choose destination=File", and "Format=FASTA", and start the download by clicking "Create file".

Check out the content of the directory containing the prepared FASTA files. You will also find a FASTA file containing Ken's 1st OTU:

`ls kaust_bioinfo_workshop_files/blast_db_setup`

Print and inspect the prepared BLAST sbatch script:

`cat kaust_bioinfo_workshop_blast_sbatch.sh`

Submit the job:

`sbatch kaust_bioinfo_workshop_blast_sbatch.sh`

And let's investigate the output together:

`cat blast_output.tsv`

Notice anything? Open the other blast output generated:

`cat blast_modified_output.tsv`

**<u>Important!</u>** All of this can be done via the [BLAST website](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastn&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome) as well - for a single sequence, i.e., one OTU. However, the advantage of the command line-based approach is that you can blast as many sequences as you want at once against the database, so potentially 1,000s of OTUs!

## 6. EXTRA: Krona graphs

Let's turn the taxonomic data of Ken's samples into a Krona graph. We need to format the data to do so. I generated a script to generate the right format. The script is written in bash to ensure that everyone can run it on IBEX, but bash is not the best coding language for such tasks. If you need to format a file on your own, you can also use R, python, or even Excel to achieve the desired format, which might be more intuitive. For now, run the script like so (we specify the path to the file we want to format and the sample ID prefix to tell the code which columns in the file are samples, in our case that is "LF"):

`./krona_formatting.sh kaust_bioinformatics_workshop_apscale/9_lulu_filtering/otu_clustering/kaust_bioinformatics_workshop_apscale_OTU_table_filtered_microdecon-filtered_with_taxonomy.csv LF`

The generated, formatted file is called "krona_formatted_file.tsv". Let's have a look at the first few lines:

`head krona_formatted_file.tsv`

Now, activate the krona mamba environment and run the krona script that turns a text file into a krona graph:

`mamba activate krona && ktImportText -o krona.html krona_formatted_file.tsv && mamba deactivate`

Download the generated file "krona.html" with Cyberduck and let's inspect it.

## 7. EXTRA: ChatGPT and Copilot for coding

Type into ChatGPT:

"You're an experienced bioinformatician who is generating an R script for a bioinformatic workshop on metabarcoding data processing. Write an R script that uses phyloseq to explore the structure of an OTU table and generates multiple different nice visualizations."

Type:

"In a phyloseq object in R, I want to remove a few OTUs by name (for example, OTU_1). How?"

## 8. EXTRA: DADA2 on IBEX

Running DADA2 on IBEX is a little bit more tricky than running, for example, Apscale, as you need to install DADA2 in your local R directory and need to setup an R script that you then upload to IBEX and run via SBATCH. First, load the R module, then generate a directory in which installed R packages are saved, and open R:

```
module load R
mkdir -p $R_LIBS
R
```

In R, type the following line by line to install dada2 and some other required packages:

```
install.packages("BiocManager") # Select mirror "1"
BiocManager::install("dada2")
BiocManager::install("phyloseq")
```

Locate the R file "dada2_master_script.R" in the kaust_bioinfo_workshop GitHub directory, open it with R Studio, and adjust the initial variables as required, which means you need to specify the path on IBEX to your sequences etc. Note: the script requires reads that were already primer-trimmed with the tool "cutadapt", so you have to run cutadapt before running the dada2 script. This means you need to install cutadapt in a mamba environment and use it to trim the primers of your raw sequences like shown in [this](https://cutadapt.readthedocs.io/en/stable/guide.html) user guide.

Then, upload the modified dada2_master_script.R script to IBEX. In the same folder, upload the SLURM script "dada2_sbatch.sh". Finally, on IBEX, submit the SLURM script via `sbatch dada2_sbatch.sh`. IMPORTANT: The R and SLURM script have to be in the same location and all your variables specified in the R script need to be correct, otherwise it won't work.